<a href="https://colab.research.google.com/github/LarissaCarrara/Gramart/blob/main/Gramart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%pip -q install google-genai

In [3]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [4]:
# Instalar Framework de agentes do Google
!pip install -q google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0

In [5]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [6]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [7]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [12]:
##########################################
# --- Agente: Busca na internet a explicação da estrutura gramatical --- #
##########################################
def agente_buscador(frase):

    buscador = Agent(
        name="agente_buscador",
        model="gemini-2.0-flash",
        instruction="""
       Você é um Professor de Português especialista em gramática. Sua função é analisar detalhadamente uma frase fornecida pelo usuário e identificar todas as suas estruturas sintáticas, explicando cada uma de forma clara e didática.

Para cada frase que eu te der, você DEVE:

Dividir a frase em Oração(ões): Se a frase tiver mais de uma oração, identifique-as separadamente.

Para cada oração, identifique o Sujeito e o Predicado.
Detalhar cada Estrutura: Para cada oração e termo, explique:

Sujeito: Indique qual é o sujeito e seu tipo (ex: simples, composto, oculto/desinencial, indeterminado, ou oração sem sujeito).
Predicado: Indique qual é o predicado e seu tipo (ex: verbal, nominal ou verbo-nominal).
Verbos: Identifique os verbos e sua função.
Complementos Verbais: Encontre Objeto Direto e Objeto Indireto.
Complementos Nominais: Identifique se houver.
Adjuntos Adnominais: Quais palavras acompanham e caracterizam os substantivos.
Adjuntos Adverbiais: Quais palavras ou expressões indicam circunstâncias (tempo, lugar, modo, causa, etc.).
Aposto: Se um termo explica ou resume outro.
Vocativo: Se houver um termo para chamar.
Conjunções/Conectivos: Se houver, explique a função de ligação entre orações ou termos.
Formato da Resposta: Apresente a análise de forma organizada e fácil de ler, usando listas ou subtítulos. Para cada estrutura identificada, mostre a parte da frase que a representa e, se aplicável, uma breve explicação ou justificativa de sua classificação.
        """,
        description="Agente para explicar a estrutura gramatical das frases",
        tools=[google_search]
    )

    entrada_do_agente_buscador = f"Frase para passar explicar a estrutura gramatical: {frase}"

    estrutura = call_agent(buscador, entrada_do_agente_buscador)
    return estrutura

In [11]:
frase = input("Qual palavra/frase você gostaria de entender a estrutura?: ")
result = agente_buscador(frase)
print("\n--- 📝 Resultado do Agente (Buscador) ---\n")
display(to_markdown(result))

Qual palavra/frase você gostaria de entender a estrutura?: Bem vindo a imersão 2025

--- 📝 Resultado do Agente 1 (Buscador) ---



> Aqui está a análise gramatical da frase "Bem-vindo à imersão 2025":
> 
> **1. Análise Geral**
> 
> *   A frase possui uma única oração.
> 
> **2. Estrutura da Oração**
> 
> *   **Sujeito:** Oculto/Desinencial (nós/vocês - depende do contexto, mas geralmente "vocês" em saudações).
> *   **Predicado:** Nominal
> 
> **3. Detalhamento**
> 
> *   **Bem-vindo:** Predicativo do sujeito (qualidade/estado atribuído ao sujeito). É a parte nominal do predicado.
> *   **à:** Contração da preposição "a" com o artigo definido feminino "a".
> *   **imersão:** Substantivo feminino, núcleo do complemento nominal.
> *   **2025:** Adjunto adnominal restritivo, especificando qual imersão (a do ano de 2025).
> 
> **4. Explicações Adicionais**
> 
> *   **Predicado Nominal:** O predicado é nominal porque o verbo (implícito, "sejam" ou "estejam") é de ligação, unindo o sujeito a uma característica (o predicativo do sujeito, "bem-vindo").
> *   **Adjunto Adnominal:** "2025" acompanha o substantivo "imersão", especificando-o.
> 
> Em resumo:
> 
> *   **Sujeito:** (Vocês)
> *   **Verbo (implícito):** Sejam/Estejam
> *   **Predicativo do Sujeito:** Bem-vindos
> *   **Complemento Nominal:** à imersão 2025 (especifica a que o "bem-vindos" se refere)
> 
> Espero que esta análise detalhada ajude na sua tradução para Libras!
> 
